In [29]:
from utils import Preprocessing, Modelling
import pandas as pd
import numpy as np

In [8]:
df = pd.read_csv('ds01_withStress.csv')
df

,Elapsed time,Driver,ECG,EMG,foot GSR,hand GSR,HR,marker,RESP,stress
0,'0:00.000',06,-0.048,0.124,9.051,19.072,90.0,12.36,39.97,1
1,'0:00.002',06,-0.052,0.124,9.051,19.072,90.0,12.36,39.97,1
2,'0:00.004',06,-0.057,0.124,9.051,19.072,90.0,12.36,39.97,1
3,'0:00.006',06,-0.063,0.124,9.051,19.072,90.0,12.36,39.97,1
4,'0:00.008',06,-0.067,0.124,9.051,19.072,90.0,12.36,39.97,1
...,...,...,...,...,...,...,...,...,...,...
48040,'0:09.990',17b,-0.022,0.124,5.322,8.557,63.0,10.15,37.48,-1
48041,'0:09.992',17b,-0.029,0.124,5.322,8.557,63.0,10.15,37.48,-1
48042,'0:09.994',17b,-0.029,0.124,5.322,8.557,63.0,10.15,37.48,-1
48043,'0:09.996',17b,-0.016,0.124,5.322,8.557,63.0,10.15,37.48,-1


In [7]:
# utils = Preprocessing(df)
# df = utils.ECGfeatures(df)
# df = utils.GSRfeatures(df)
# df

In [11]:
import pandas as pd
df = pd.DataFrame(columns =['A', 'B'])
# add = pd.DataFrame([[]])
df1 = pd.DataFrame({'A': [0 for _ in range(5)], 'B':[2 for _ in range(5)]})
df = df.append(df1)
df

,A,B
0,0,2
1,0,2
2,0,2
3,0,2
4,0,2


In [ ]:
# encoded_df = []
# decoded_df = []
def enc_columns(df, steps = 2000, epochs = 50):
    cols =  ['ECG', 'EMG', 'foot GSR', 'hand GSR', 'HR', 'RESP']
    
    for col in cols:
        enc_df = pd.DataFrame(columns= ['Driver',col + ' Encoded'])
        driver = df['Driver'].value_counts().keys()
        df[col+' CAE'] = [-1 for _ in range(len(df))]
        for dr in driver:
            start_index = df.loc[df['Driver']==dr, col].index[0]
            train_CAE = []
            encoding_series = df.loc[df['Driver']==dr, col]
            for val in range(steps, len(encoding_series), steps):
                temp = np.array(encoding_series[val-steps: val]).reshape((steps, 1))
                train_CAE.append([temp])

            train_CAE = np.array(train_CAE)
            train_CAE = train_CAE.reshape(-1, steps, 1)
            print(train_CAE.shape)

            mdl, enc = Modelling.CAE(steps)

            mdl.fit(train_CAE, train_CAE.reshape(-1, steps), epochs = epochs, shuffle = False)
            dec_CAE = mdl.predict(train_CAE)
            mdl.save(col + 'CAE Model.h5')
            dec_CAE = dec_CAE.reshape(train_CAE.shape[0] * train_CAE.shape[1])
            dec_CAE = list(dec_CAE)
            end_index = start_index + len(dec_CAE) - 1
        #     filtered_ecg = filtered = ecg.ecg(ecg_CAE, sampling_rate = 500, show = False)
            print(len(dec_CAE), start_index, start_index+len(dec_CAE))
            df.loc[start_index:start_index+len(dec_CAE)-1, col+' CAE'] = dec_CAE #filtered_ecg['filtered']

            enc_CAE= enc.predict(train_CAE)
            enc_CAE = enc_CAE.reshape(63,)
            
            enc_df.append({'Driver':[dr for_ in range(len(enc_CAE))], col+' Encoded':[val for _ in enc_CAE]})

            print('*'*5, col+':' + dr, 'Done')
            
        enc_df.to_csv(col+' encoded.csv', index = False)
            
    return df

enc_columns(df)

(2, 2000, 1)
Epoch 1/1
2/2 [==============================] - 17s 9s/step - loss: 0.3234
4000 43085 47085
17b Done
(2, 2000, 1)
Epoch 1/1
2/2 [==============================] - 13s 7s/step - loss: 0.0431
4000 4960 8960
07 Done
(2, 2000, 1)
Epoch 1/1
2/2 [==============================] - 23s 11s/step - loss: 0.0478
4000 14880 18880
09 Done
(2, 2000, 1)
Epoch 1/1
2/2 [==============================] - 14s 7s/step - loss: 0.0444
4000 24800 28800
11 Done
(2, 2000, 1)
Epoch 1/1
2/2 [==============================] - 28s 14s/step - loss: 0.0374
4000 9920 13920
08 Done
(2, 2000, 1)
Epoch 1/1
2/2 [==============================] - 31s 16s/step - loss: 0.0272
4000 33325 37325
16 Done
(2, 2000, 1)
Epoch 1/1
2/2 [==============================] - 15s 8s/step - loss: 0.0187
4000 19840 23840
10 Done
(2, 2000, 1)
Epoch 1/1
2/2 [==============================] - 29s 14s/step - loss: 0.0313
4000 0 4000
06 Done
(2, 2000, 1)
Epoch 1/1
2/2 [==============================] - 20s 10s/step - loss: 0.0368
4

In [23]:
models = Modelling()
CAE = models.CAE()
cols_enc = ['ECG', 'EMG', 'foot GSR', 'hand GSR', 'HR', 'RESP']
df_enc = pd.DataFrame(columns = cols_enc+['Driver'])
for col in cols_enc:
    series = df.loc[:, ['Driver']+[col]]
    ecg_CAE, dec_CAE, start_index, end_index = enc_column(df_enc, col) 

UnboundLocalError: local variable 'enc_CAE' referenced before assignment

In [38]:
len(df)

48045